In [1]:
pip install transformers

In [2]:
pip install bitsandbytes

In [3]:
pip install accelerate

In [4]:
#Colab Google Drive SetUp
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
#Colab Access Token SetUp
from google.colab import userdata
access_token = userdata.get('HF_TOKEN')

In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
from statsmodels.stats.contingency_tables import mcnemar

In [7]:
from scipy import stats
from itertools import combinations

In [8]:
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import BitsAndBytesConfig
import torch

In [9]:
import os

In [10]:
#df_anno = pd.read_csv('DH_2025_Anno_DEU_agreed.csv')
df_anno = pd.read_csv('/content/drive/MyDrive/Reengineering_Focalization/DH_2025_Anno_DEU_2nd_run.csv')

In [11]:
df_anno['Fokalisierung'] = df_anno['Fokalisierung'].replace({
    'intern': 'internal',
    'extern': 'external',
    'null': 'zero',
    None: 'zero',
    np.nan: 'zero'
})

In [12]:
df_anno.head()

,Autor,Titel,Absatz,Fokalisierung,Kommentar
0,Schiller,Der Vebrecher aus verlorener Ehre,In der ganzen Geschichte des Menschen ist kein...,zero,NaN
1,Schiller,Der Vebrecher aus verlorener Ehre,Es ist etwas so Einförmiges und doch wieder so...,zero,NaN
2,Schiller,Der Vebrecher aus verlorener Ehre,"Ich weiß, daß von den besten Geschichtschreibe...",internal,NaN
3,Schiller,Der Vebrecher aus verlorener Ehre,"Der Held muß kalt werden wie der Leser, oder, ...",zero,NaN
4,Tieck,Die beiden merkwürdigsten Tage aus Siegmunds L...,"Es war schon gegen Abend, als ein Wagen vor de...",internal,NaN


In [13]:
model_id = "google/gemma-2-9b-it" #Fill in the name of the choosen model
tokenizer = AutoTokenizer.from_pretrained(model_id, token=access_token, use_fast=True)

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

In [14]:
quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16
    )

In [15]:
#model_16bit = AutoModelForCausalLM.from_pretrained(
    #model_id,
    #token=access_token,
    #device_map="auto",
    #quantization_config=quantization_config)

In [16]:
generate_text = pipeline(
            model=model_id,
            tokenizer=tokenizer,
            device_map="auto",
            return_full_text=False,  # if using langchain set True
            do_sample=True,
            task="text-generation",
            max_new_tokens=512,
            temperature=0.1
        )

config.json:   0%|          | 0.00/857 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/39.1k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [17]:
prompt_basic = """
### Instruction
Your task is to classify the focalization of the following sentence

###
Only respond with one word representing the mode of focalization, do NOT give explenations or generate more text
Sentence: '''{text}''''
Label:
"""

In [18]:
prompt_labels = """
### Instruction
Your task is to classify the focalization of the following sentence

### Labels
There are three modes of focalization:
- internal
- external
- zero

####
Only respond with one word representing the mode of focalization, do NOT give explenations or generate more text
Sentence: '''{text}'''
Label:
"""

In [19]:
prompt_redefin = """
### Instruction
Your task is to classify the focalization of the following sentence

### Labels
There are three modes of focalization:
- internal: A text passage is internally focalized precisely when a perceptual process is part of the depicted event and is presented from the perspective of a character.
- external: A text passage is externally focalized precisely when a perceptual process is part of the depicted event and could be presented from the perspective of a character.
- zero: A text passage is zero focalized precisely when circumstances of the narrated world are described as if they were independent of a particular perceptual process of a person or are not possible for a person to perceive synchronously.

####
Only respond with one word representing the mode of focalization, do NOT give explenations or generate more text
Sentence: '''{text}'''
Label:
"""

In [20]:
prompt_meta = """
### Instruction
Your task is to classify the focalization of the following sentence

### Labels
There are three modes of focalization:
- internal: A text passage is internally focalized precisely when a perceptual process is part of the depicted event and is presented from the perspective of a character.
- external: A text passage is externally focalized precisely when a perceptual process is part of the depicted event and could be presented from the perspective of a character.
- zero: A text passage is zero focalized precisely when circumstances of the narrated world are described as if they were independent of a particular perceptual process of a person or are not possible for a person to perceive synchronously.
These definitions are redefinitions of the standard understanding of focalization.

####
Only respond with one word representing the mode of focalization, do NOT give explenations or generate more text
Sentence: '''{text}'''
Label:
"""

In [21]:
prompt_templates = [prompt_labels, prompt_redefin, prompt_meta]

In [22]:
def evaluate_prompts_and_predictions(df, prompt_templates):
    """
    Evaluiert verschiedene Prompt-Templates und berechnet Metriken für die Vorhersagen.

    Args:
        df: Pandas DataFrame mit den Spalten 'Absatz' und 'Fokalisierung'
        prompt_templates: Liste der Prompt-Templates

    Returns:
        DataFrame mit den Evaluierungsmetriken für jeden Prompt
    """

    results = []

    # Iteration über die Prompt-Templates
    for prompt_idx, template in enumerate(prompt_templates):
        print(f"Verarbeite Prompt-Template {prompt_idx + 1}/{len(prompt_templates)}")

        # Neue Spalte für Vorhersagen erstellen
        df[f'Prediction_{prompt_idx}'] = None

        # Iteration über die Zeilen des DataFrames
        for idx, row in df.iterrows():
            # Prompt erstellen durch Einsetzen des Absatzes in das Template
            prompt = template.format(text=row['Absatz'])

            # Vorhersage mit Mixtral-Modell
            result = generate_text(prompt)
            output = result[0]["generated_text"]
            first_word = output.split()[0]
            first_word = first_word.lower()
            print(first_word)

            # Vorhersage speichern
            df.at[idx, f'Prediction_{prompt_idx}'] = first_word

        #print(df)

        # Metriken berechnen
        metrics = {
            'Prompt': f'Template_{prompt_idx}',
            'F1-Score': f1_score(df['Fokalisierung'], df[f'Prediction_{prompt_idx}'], average='weighted'),
            'Recall': recall_score(df['Fokalisierung'], df[f'Prediction_{prompt_idx}'], average='weighted'),
            'Precision': precision_score(df['Fokalisierung'], df[f'Prediction_{prompt_idx}'], average='weighted'),
            'Accuracy': accuracy_score(df['Fokalisierung'], df[f'Prediction_{prompt_idx}'])
        }

        results.append(metrics)

    # Ergebnisse in DataFrame umwandeln
    results_df = pd.DataFrame(results)

    return df, results_df

In [23]:
results_gemma, test = evaluate_prompts_and_predictions(df_anno, prompt_templates)

Verarbeite Prompt-Template 1/3
external
external
external
external
external
external
internal
internal
internal


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


external
internal
internal
internal
internal
internal
external
external
external
external
internal
internal
internal
internal
internal
external
external
external
internal
external
external
internal
external
internal
internal
internal
internal
external
external
external
external
internal
internal
internal
internal
internal
internal
internal
internal
external
external
external
internal
internal
internal
internal
internal
external
external
external
external
external
external
external
external
internal
external
external
external
external
external
internal
external
external
external
external
internal
internal
external
internal
internal
internal
internal
external
external
external
external
external
internal
external
internal
internal
external
external
internal
external
internal
internal
internal
external
external
internal
external
external
internal
external
external
Verarbeite Prompt-Template 2/3


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


external
external
external
external
external
external
internal
internal
internal
external
internal
external
internal
external
internal
external
external
external
external
internal
internal
internal
internal
external
external
external
external
internal
external
external
external
external
internal
external
internal
internal
external
external
external
external
internal
internal
internal
internal
internal
internal
internal
internal
external
external
external
external
external
external
external
internal
external
external
external
external
external
external
external
external
external
external
external
external
external
external
internal
external
external
external
external
internal
internal
external
external
internal
external
internal
external
external
external
external
external
internal
external
external
external
external
external
external
external
internal
internal
internal
external
zero
internal
external
external
internal
external
external
Verarbeite Prompt-Template 3/3
external
external
e

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
test

,Prompt,F1-Score,Recall,Precision,Accuracy
0,Template_0,0.450290,0.490566,0.451047,0.490566
1,Template_1,0.415661,0.443396,0.492790,0.443396
2,Template_2,0.397130,0.424528,0.485985,0.424528


## Statistischer Test 1: McNemar

In [25]:
# Funktion zum Vergleichen der Vorhersagen mit dem Goldstandard
def compare_with_gold(predictions, gold_standard):
    return [pred == gold for pred, gold in zip(predictions, gold_standard)]

In [26]:
results_1 = compare_with_gold(results_gemma["Prediction_0"], df_anno["Fokalisierung"])
results_2 = compare_with_gold(results_gemma["Prediction_1"], df_anno["Fokalisierung"])
results_3 = compare_with_gold(results_gemma["Prediction_2"], df_anno["Fokalisierung"])

In [27]:
def create_contingency_table(results_a, results_b):
    both_correct = sum(a and b for a, b in zip(results_a, results_b))
    only_a_correct = sum(a and not b for a, b in zip(results_a, results_b))
    only_b_correct = sum(b and not a for a, b in zip(results_a, results_b))
    both_incorrect = sum(not a and not b for a, b in zip(results_a, results_b))
    return np.array([[both_correct, only_a_correct],
                     [only_b_correct, both_incorrect]])

In [28]:
def run_mcnemar_test(results_a, results_b):
    table = create_contingency_table(results_a, results_b)
    return mcnemar(table, exact=True)

In [29]:
test_1_2 = run_mcnemar_test(results_1, results_2)
test_1_3 = run_mcnemar_test(results_1, results_3)
test_2_3 = run_mcnemar_test(results_2, results_3)

In [30]:
print("McNemar-Test Ergebnisse:")
print(f"Template 1 vs Template 2: p-Wert = {test_1_2.pvalue}")
print(f"Template 1 vs Template 3: p-Wert = {test_1_3.pvalue}")
print(f"Template 2 vs Template 3: p-Wert = {test_2_3.pvalue}")

McNemar-Test Ergebnisse:
Template 1 vs Template 2: p-Wert = 0.266845703125
Template 1 vs Template 3: p-Wert = 0.11846923828124999
Template 2 vs Template 3: p-Wert = 0.5


## Statistischer Test 2: Paired-ttest

In [31]:
def analyze_statistical_significance(df, results_df):
    """
    Analysiert die statistische Signifikanz der Unterschiede zwischen F1-Scores verschiedener Prompts

    Args:
        df: Original DataFrame mit den Vorhersagen
        results_df: DataFrame mit den Evaluierungsmetriken aus evaluate_prompts_and_predictions

    Returns:
        DataFrame mit den Ergebnissen der statistischen Tests
    """

    # Liste aller Prompt-Kombinationen erstellen
    prompt_combinations = list(combinations(results_df['Prompt'], 2))

    # Ergebnisse speichern
    test_results = []

    for prompt1, prompt2 in prompt_combinations:
        # Indizes extrahieren
        idx1 = int(prompt1.split('_')[1])
        idx2 = int(prompt2.split('_')[1])

        # F1-Scores für jeden einzelnen Fall berechnen
        f1_scores_1 = []
        f1_scores_2 = []

        # Für jede Instanz einzeln F1-Score berechnen
        for idx in df.index:
            true = df.loc[idx, 'Fokalisierung']
            pred1 = df.loc[idx, f'Prediction_{idx1}']
            pred2 = df.loc[idx, f'Prediction_{idx2}']

            # Einzelne F1-Scores berechnen (1 für korrekt, 0 für falsch)
            f1_scores_1.append(1 if true == pred1 else 0)
            f1_scores_2.append(1 if true == pred2 else 0)

        # Verschiedene statistische Tests durchführen

        # 1. Paired t-test
        t_stat, t_pvalue = stats.ttest_rel(f1_scores_1, f1_scores_2)

        # 2. Wilcoxon signed-rank test
        w_stat, w_pvalue = stats.wilcoxon(f1_scores_1, f1_scores_2)

        # Mittlere Differenz berechnen
        mean_diff = np.mean(f1_scores_1) - np.mean(f1_scores_2)

        # Ergebnisse speichern
        test_results.append({
            'Prompt_Comparison': f'{prompt1} vs {prompt2}',
            'Mean_Difference': mean_diff,
            'T_Test_p_value': t_pvalue,
            'Wilcoxon_p_value': w_pvalue,
            'Significant_0.05': any(p < 0.05 for p in [t_pvalue, w_pvalue]),
            'Significant_0.01': any(p < 0.01 for p in [t_pvalue, w_pvalue])
        })

    # Ergebnisse in DataFrame umwandeln
    results_df = pd.DataFrame(test_results)

    return results_df

In [32]:
stats = analyze_statistical_significance(df_anno, test)

In [33]:
stats

,Prompt_Comparison,Mean_Difference,T_Test_p_value,Wilcoxon_p_value,Significant_0.05,Significant_0.01
0,Template_0 vs Template_1,0.047170,0.166596,0.165518,False,False
1,Template_0 vs Template_2,0.066038,0.070507,0.070701,False,False
2,Template_1 vs Template_2,0.018868,0.158281,0.157299,False,False
